In [1]:
import os
os.chdir("../")

In [2]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from anidex.constants import *
from anidex.utils.common import read_yaml, create_directories

In [7]:
from anidex.constants import  CONFIG_FILE_PATH
CONFIG_FILE_PATH

WindowsPath('config/config.yaml')

In [8]:
print(CONFIG_FILE_PATH)

config\config.yaml


In [10]:
read_yaml(CONFIG_FILE_PATH)

[2024-04-02 16:47:17,726: INFO: common: yaml file: config\config.yaml loaded successfully]


ConfigBox({'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/11_JAlknRN1wgNmVhwWumn8IkgXm_4phD/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_preprocessing': {'root_dir': 'artifacts/data_preprocessing', 'dataset_path': 'artifacts/data_ingestion/Dataset.csv', 'preprocessor_path': 'artifacts/data_preprocessing/preprocessor.h5', 'tracked_preprocessor_path': 'preprocessor/preprocessor.pkl'}, 'training': {'root_dir': 'artifacts/training', 'training_data': 'artifacts/processed_data.csv', 'trained_model_file_path': 'artifacts/training/model.h5', 'trained_model_file_path_rent': 'artifacts/training/model_rent.h5', 'mlflow_uri': 'https://dagshub.com/Rajarshi12321/My-Sweet-Home.mlflow'}, 'recommed_data_preprocessing': {'root_dir': 'artifacts/recommed', 'dataset_path': 'artifacts/data_ingestion/Dataset.csv', 'processed_dataset_path': 'ar

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os
import zipfile
import gdown
from anidex import logger
from anidex.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2024-04-02 16:48:07,225: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-02 16:48:07,242: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-02 16:48:07,246: INFO: common: created directory at: artifacts]
[2024-04-02 16:48:07,251: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-02 16:48:07,254: INFO: 1579108944: Downloading data from https://drive.google.com/file/d/11_JAlknRN1wgNmVhwWumn8IkgXm_4phD/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=11_JAlknRN1wgNmVhwWumn8IkgXm_4phD
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=11_JAlknRN1wgNmVhwWumn8IkgXm_4phD&confirm=t&uuid=05a68d4e-a7ed-4888-9710-2ad71c9a75c1
To: e:\projects\anidex\artifacts\data_ingestion\data.zip
100%|██████████| 688M/688M [01:20<00:00, 8.50MB/s] 

[2024-04-02 16:49:31,192: INFO: 1579108944: Downloaded data from https://drive.google.com/file/d/11_JAlknRN1wgNmVhwWumn8IkgXm_4phD/view?usp=sharing into file artifacts/data_ingestion/data.zip]
